In [1]:
#import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import text_to_word_sequence
import re
import string
stopWords = set(stopwords.words('english'))


Using TensorFlow backend.


In [2]:

import gensim
google_model_path= 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
# Load Google's pre-trained Word2Vec model.
# model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  
model = gensim.models.KeyedVectors.load_word2vec_format(google_model_path, binary=True, limit=10 ** 5)

In [3]:
df_original = pd.read_csv('../OnlineHarassmentDataset/onlineHarassmentDataset.tdf',
                          sep='\t', 
                          lineterminator='\r',
                          header = 'infer', 
                          encoding='latin-1')
df=df_original.copy(deep = True)

In [4]:
df.head()

,ID,Code,Tweet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,H,@Grumpy_P_Sloth @deanesmay feminists argue for...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,H,1948Army of England helped the Jews to occupy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,H,Dutch Leader Says Europe to collapse In 6 Week...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,H,RT @__DeLay: The next day the Romans and the J...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,H,RT @Bakersman_Joe: When Hitler Invited The Jew...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# How balanced are the classes?
print('Harassment is ' + str(round(sum(df['Code'] == 'H')/df.shape[0],4) * 100) + '% of the dataset')

Harassment is 25.96% of the dataset


In [6]:
# Quality of Life renaming
df = df[['Code','Tweet']]
df.rename(index=str, columns={"Code": "code", "Tweet": "tweet"}, inplace = True)

In [7]:
df.iloc[200,1]

'@azhodzic lmfaaaooooo @jeffbarnes_  you fucking nigger'

# List of Regex Strings to Remove

In [8]:
df['tweet'] = df['tweet'].apply(lambda tweet: tweet.lower())

In [9]:
regex_queries = [
                 '@([A-Za-z0-9_]+)', #Usernames
                'rt\s:\s', #Retweets
                 '(https?:\s?\/\s?\/)(\s)?(www\.)?(\s?)(\w+\.)*([\w\-\s]+\/)*([\w-]+)\/?' # Hyperlinks
                ]

In [10]:
df['tweet'].replace(regex_queries,'',regex = True, inplace = True)
# Remove symbols, replace with spaces
df['tweet'].replace('[^\w\s]|_','', regex = True,inplace = True)
# df['tweet'].replace('/\s+/g','', regex = True,inplace = True)

In [11]:
print(df_original['Tweet'][:10].values)

["@Grumpy_P_Sloth @deanesmay feminists argue for raising minimum wage cause a women's studies major needs the $15 an hour wage at her McJob."
 '1948Army of England helped the Jews to occupy Palestine.After that the Jews killed thousands of Englishmen and returned to England as dogs1'
 'Dutch Leader Says Europe to collapse In 6 Weeks thnks > @BillGates, @GeorgeSoros, & @FWD_us #AltRight #WhiteGenocide https: / /t.co /rl1mh82jda'
 'RT @__DeLay: The next day the Romans and the Jews or watever was like "aye fuck that nigga Jesus. We riding out on his ass today https: / /t.'
 "RT @Bakersman_Joe: When Hitler Invited The Jews To The Smoke Session But It Wasn't The Gas They Was Expecting #MemeHistory https: / /t.co /lMw"
 'RT @billmaher: Bravo to #Netanyahu for his 45 second glare this week at the UN! The Jews have finally done it, they weaponized guilt!'
 'RT @BitchImKermit: White people literally Be like " I\'m not racist you fucking Nigger"'
 'RT @CharlesFinch: "The Jews were provided free 

In [12]:
print(df['tweet'][:10].values)

['  feminists argue for raising minimum wage cause a womens studies major needs the 15 an hour wage at her mcjob'
 '1948army of england helped the jews to occupy palestineafter that the jews killed thousands of englishmen and returned to england as dogs1'
 'dutch leader says europe to collapse in 6 weeks thnks      altright whitegenocide '
 'the next day the romans and the jews or watever was like aye fuck that nigga jesus we riding out on his ass today '
 'when hitler invited the jews to the smoke session but it wasnt the gas they was expecting memehistory '
 'bravo to netanyahu for his 45 second glare this week at the un the jews have finally done it they weaponized guilt'
 'white people literally be like  im not racist you fucking nigger'
 'the jews were provided free transportation directly to the camps  bill oreilly'
 'i would have killed all the jews in the world but i kept some to show the world why i killed them hitler  '
 '  i can do news i can do newspeople and i am news when

In [13]:
df_original['Tweet'].iloc[9]

"RT @HeymanHustle: .. @MichelleDBeadle I can do news. I can do newspeople. And I AM news. When they say THE JEWS control the media, they're "

In [14]:
df['tweet'].iloc[2]

'dutch leader says europe to collapse in 6 weeks thnks      altright whitegenocide '

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(stop_words='english')

X_train_cv1 = cv1.fit_transform(X_train)
X_test_cv1  = cv1.transform(X_test)

pd.DataFrame(X_train_cv1.toarray(), columns=cv1.get_feature_names()).head()

NameError: name 'X_train' is not defined

In [ ]:

data = "dutch leader says europe to collapse in 6 weeks thnks      altright whitegenocide "
#stopWords = set(stopwords.words('english'))
words = word_tokenize(data)
wordsFiltered = []

for w in words:
    if w not in stopWords:
        wordsFiltered.append(w)

print(wordsFiltered)

# Remove Punctuation

In [ ]:
# Done already

# def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
#     text = re.sub('[0-9]+', '', text)
#     return text

# df['Tweet_punct'] = df['Tweet'].apply(lambda x: remove_punct(x))
# df.head(10)

# Tokenization

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['tweet'].apply(lambda x: tokenization(x.lower()))
df.head()

# Remove Stopwords

In [ ]:

def remove_stopwords(text):
    text = [word for word in text if word not in stopWords]
    return text
    
df['Tweet_nonstop'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

# Stemming / Lemmatization

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_nonstop'].apply(lambda x: stemming(x))
df.head()

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['Tweet_lemmatized'] = df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
df.head()

In [ ]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopWords]  # remove stopwords and stemming
    return text
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(df['tweet'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))
#print(countVectorizer.get_feature_names())
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
